In [4]:
from pathlib import Path
import itertools
import pandas as pd

from chan.chan_trend import label_trend_practical


ROOT = Path.cwd().resolve()
if ROOT.name == "Note":
    ROOT = ROOT.parents[0]
CHAN = ROOT / "chan_data"

# segments 使用 fast 版本
SEG_BASE = CHAN / "segments" / "fast"
ZS_HIER = CHAN / "zhongshu_hierarchy_fast"

symbols = ["BTCUSDT", "ETHUSDT"]
tfs = ["5m","15m","30m","1h","4h"]

gap_hours_list = [12, 24, 36, 48, 72]
price_thr_list = [0.005, 0.01, 0.015, 0.02]
min_segments_list = [2, 3, 4]
enable_quality_list = [True]

def _eval_trends(trend_df: pd.DataFrame) -> dict:
    if trend_df is None or trend_df.empty:
        return dict(
            n_total=0, n_trend=0, n_range=0,
            trend_ratio=0.0, rr_med=0.0, seg_med=0.0,
            dur_med=0.0, strength_med=0.0,
            qa=0, qb=0, qc=0, time_coverage=0.0
        )
    df = trend_df.copy()
    n_total = len(df)
    n_trend = int((df["type"] == "trend").sum())
    n_range = n_total - n_trend
    trend_ratio = n_trend / max(1, n_total)

    rr_med = float(df.loc[df["type"]=="trend","rr"].median()) if n_trend>0 else 0.0
    seg_med = float(df["segment_count"].median()) if n_total>0 else 0.0
    dur_med = float(df["duration_days"].median()) if n_total>0 else 0.0
    strength_med = float(df["overall_strength"].median()) if "overall_strength" in df.columns and n_total>0 else 0.0

    qa = int((df.get("quality_grade","C")=="A").sum())
    qb = int((df.get("quality_grade","C")=="B").sum())
    qc = int((df.get("quality_grade","C")=="C").sum())

    t0 = pd.to_datetime(df["timestamp_start"]).min() if n_total>0 else pd.NaT
    t1 = pd.to_datetime(df["timestamp_end"]).max() if n_total>0 else pd.NaT
    cov = 0.0
    if n_total>0 and pd.notna(t0) and pd.notna(t1) and t1>t0:
        lens = (pd.to_datetime(df["timestamp_end"]) - pd.to_datetime(df["timestamp_start"])).dt.total_seconds()
        cov = float(lens.sum() / (t1 - t0).total_seconds())
        cov = max(0.0, min(1.5, cov))
    return dict(
        n_total=int(n_total), n_trend=int(n_trend), n_range=int(n_range),
        trend_ratio=float(trend_ratio), rr_med=float(rr_med),
        seg_med=float(seg_med), dur_med=float(dur_med),
        strength_med=float(strength_med),
        qa=int(qa), qb=int(qb), qc=int(qc), time_coverage=float(cov)
    )

def _score_metric(m):
    tr = m["trend_ratio"]
    band_penalty = abs(tr - 0.45)
    score = (
        2.0 * m["rr_med"]
        + 0.5 * m["time_coverage"]
        + 0.2 * m["strength_med"]
        - 0.8 * band_penalty
    )
    return float(score)

rows = []
missing = []

for sym, tf in itertools.product(symbols, tfs):
    seg_path = SEG_BASE / sym / f"{tf}_segments.csv"
    if not seg_path.exists():
        missing.append((sym, tf))
        continue

    seg = pd.read_csv(seg_path, parse_dates=["timestamp_start","timestamp_end"])
    if seg.empty:
        missing.append((sym, tf))
        continue

    need = {"segment_index","timestamp_start","timestamp_end","start_price","end_price","low","high"}
    if not need.issubset(seg.columns):
        missing.append((sym, tf))
        continue

    for gap_h, thr, min_k, eq in itertools.product(gap_hours_list, price_thr_list, min_segments_list, enable_quality_list):
        tr = label_trend_practical(
            seg,
            gap_hours=float(gap_h),
            price_change_threshold=float(thr),
            min_segments=int(min_k),
            enable_quality=bool(eq),
        )
        m = _eval_trends(tr)
        score = _score_metric(m)
        rows.append(dict(
            symbol=sym, tf=tf,
            gap_hours=gap_h, price_thr=thr, min_segments=min_k, enable_quality=eq,
            score=score, **m
        ))

res = pd.DataFrame(rows).sort_values(["symbol","tf","score"], ascending=[True, True, False]).reset_index(drop=True)

print("missing：", missing)
print("Total evaluation of the number of combinations：", len(res))

# 展示每个 symbol-tf 的 top3
topk = res.groupby(["symbol","tf"]).head(3).reset_index(drop=True)
display(topk)

缺失/空数据： []
共评估组合数： 600


symbol   tf  gap_hours  price_thr  min_segments  enable_quality  \
0   BTCUSDT  15m         72      0.015             2            True   
1   BTCUSDT  15m         72      0.015             3            True   
2   BTCUSDT  15m         72      0.015             4            True   
3   BTCUSDT   1h         48      0.015             2            True   
4   BTCUSDT   1h         48      0.020             2            True   
5   BTCUSDT   1h         48      0.005             3            True   
6   BTCUSDT  30m         48      0.020             3            True   
7   BTCUSDT  30m         48      0.020             2            True   
8   BTCUSDT  30m         36      0.010             2            True   
9   BTCUSDT   4h         36      0.020             2            True   
10  BTCUSDT   4h         48      0.020             2            True   
11  BTCUSDT   4h         12      0.015             2            True   
12  BTCUSDT   5m         72      0.005             2            True   
13  BTCUSDT   5m         48      0.005             2            True   
14  BTCUSDT   5m         48      0.005             3            True   
15  ETHUSDT  15m         72      0.005             2            True   
16  ETHUSDT  15m         72      0.005             3            True   
17  ETHUSDT  15m         72      0.005             4            True   
18  ETHUSDT   1h         36      0.010             3            True   
19  ETHUSDT   1h         36      0.015             3            True   
20  ETHUSDT   1h         36      0.020             3            True   
21  ETHUSDT  30m         72      0.005             2            True   
22  ETHUSDT  30m         72      0.005             3            True   
23  ETHUSDT  30m         72      0.005             4            True   
24  ETHUSDT   4h         24      0.005             3            True   
25  ETHUSDT   4h         24      0.005             4            True   
26  ETHUSDT   4h         24      0.010             3            True   
27  ETHUSDT   5m         48      0.005             2            True   
28  ETHUSDT   5m         48      0.005             3            True   
29  ETHUSDT   5m         48      0.005             4            True   

       score  n_total  n_trend  n_range  trend_ratio    rr_med  seg_med  \
0   0.457096       27        6       21     0.222222  0.100162      1.0   
1   0.443705       24        5       19     0.208333  0.104002      1.0   
2   0.443705       24        5       19     0.208333  0.104002      1.0   
3   0.462208       22       10       12     0.454545  0.040792      1.0   
4   0.462208       22       10       12     0.454545  0.040792      1.0   
5   0.416603       16        8        8     0.500000  0.040792      2.0   
6   0.516854       25       10       15     0.400000  0.051158      1.0   
7   0.495021       27       10       17     0.370370  0.051158      1.0   
8   0.491470       35       15       20     0.428571  0.042593      1.0   
9   0.510459        7        3        4     0.428571  0.150696      1.0   
10  0.510459        7        3        4     0.428571  0.150696      1.0   
11  0.476281        3        1        2     0.333333  0.037654      1.0   
12  0.398392       19        6       13     0.315789  0.042403      1.0   
13  0.396082       24       11       13     0.458333  0.027383      1.0   
14  0.360537       22        9       13     0.409091  0.027383      1.0   
15  1.558144        1        1        0     1.000000  0.649072    256.0   
16  1.558144        1        1        0     1.000000  0.649072    256.0   
17  1.558144        1        1        0     1.000000  0.649072    256.0   
18  0.658078       22       11       11     0.500000  0.153888      2.0   
19  0.653595       26       10       16     0.384615  0.160073      1.0   
20  0.653595       26       10       16     0.384615  0.160073      1.0   
21  0.706398        3        3        0     1.000000  0.231872     47.0   
22  0.706398        3        3        0     1.0

In [6]:
res.to_csv(ROOT / "reports" / "trend_practical_grid.csv", index=False)
res.groupby(["symbol","tf"]).head(1).to_csv(
    ROOT / "reports" / "trend_practical_best.csv", index=False)

In [7]:
from pathlib import Path
import pandas as pd
from chan.chan_trend import label_trend_practical

ROOT = Path.cwd().resolve()
if ROOT.name == "Note":
    ROOT = ROOT.parents[0]
CHAN = ROOT / "chan_data"
SEG_DIR = CHAN / "segments" / "fast"
OUT_DIR = CHAN / "trend_practical"
OUT_DIR.mkdir(parents=True, exist_ok=True)

best_path = ROOT / "reports" / "trend_practical_best.csv"
best = pd.read_csv(best_path)

for _, row in best.iterrows():
    sym, tf = row["symbol"], row["tf"]
    p_seg = SEG_DIR / sym / f"{tf}_segments.csv"
    if not p_seg.exists():
        print(f"[skip] {sym}-{tf}: segments missing")
        continue

    seg = pd.read_csv(p_seg, parse_dates=["timestamp_start","timestamp_end"])
    if seg.empty:
        print(f"[skip] {sym}-{tf}: empty")
        continue

    print(f"[run] {sym}-{tf} ...")
    tr = label_trend_practical(
        seg,
        gap_hours=float(row["gap_hours"]),
        price_change_threshold=float(row["price_thr"]),
        min_segments=int(row["min_segments"]),
        enable_quality=True
    )

    outp = OUT_DIR / sym
    outp.mkdir(parents=True, exist_ok=True)
    tr.to_csv(outp / f"{tf}_trend.csv", index=False)
    print(f"[saved] {sym}-{tf}: n={len(tr)}")

print("[done] all trend files saved.")

[run] BTCUSDT-15m ...
[saved] BTCUSDT-15m: n=27
[run] BTCUSDT-1h ...
[saved] BTCUSDT-1h: n=22
[run] BTCUSDT-30m ...
[saved] BTCUSDT-30m: n=25
[run] BTCUSDT-4h ...
[saved] BTCUSDT-4h: n=7
[run] BTCUSDT-5m ...
[saved] BTCUSDT-5m: n=19
[run] ETHUSDT-15m ...
[saved] ETHUSDT-15m: n=1
[run] ETHUSDT-1h ...
[saved] ETHUSDT-1h: n=22
[run] ETHUSDT-30m ...
[saved] ETHUSDT-30m: n=3
[run] ETHUSDT-4h ...
[saved] ETHUSDT-4h: n=1
[run] ETHUSDT-5m ...
[saved] ETHUSDT-5m: n=5
[done] all trend files saved.
